In [1]:
from lungstage.dash import show_app

ImportError: No module named 'lungstage'

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from plotly import graph_objs as go
from dash.dependencies import Input, Output

In [ ]:
test_dash_app = dash.Dash(__name__, url_base_pathname='/', csrf_protect=False)
test_dash_app.layout = html.Div([dcc.RadioItems(id='item_list', 
                                                options = [dict(label = k, value = k) for k in ['Hey', 'Bob']]), 
                                 dcc.RadioItems(id='subitem_list', value = [])])
@test_dash_app.callback(
    Output(component_id='subitem_list', component_property='options'),
    [Input(component_id='item_list', component_property='value')]
)
def update_lesion_list(selected_idx):
    return [{'label':  '<img src="https://dummyimage.com/%i.jpg">hey</img>' % (100+i), 'value': i} for i, lab_name in enumerate('abcde')]


In [ ]:
show_app(test_dash_app)

In [ ]:
test_dash_app = dash.Dash(__name__, url_base_pathname='/', csrf_protect=False)
test_dash_app.layout = html.Div([dcc.RadioItems(id='item_list', 
                                                options = [dict(label = k, value = k) for k in ['Hey', 'Bob']]), 
                                 html.Div(id='button_list')])
@test_dash_app.callback(
    Output(component_id='button_list', component_property='children'),
    [Input(component_id='item_list', component_property='value')]
)
def update_button_list(selected_idx):
    if selected_idx is not None:
        return [html.Button('Hey %04d' % (i), id = 'id_%s_%04d' % (selected_idx, i)) for i in range(4)]

In [ ]:
show_app(test_dash_app)

In [ ]:
test_dash_app = dash.Dash(__name__, url_base_pathname='/', csrf_protect=False)
test_dash_app.layout = html.Div([dcc.RadioItems(id='item_list', 
                                                options = [dict(label = k, value = k) for k in ['Hey', 'Bob']]), 
                                 html.Div(id='button_list')])

def fancy_button_adder(*args):
    return [html.P(y) for arg in args]

@test_dash_app.callback(
    Output(component_id='button_list', component_property='children'),
    [Input(component_id='item_list', component_property='value')]
)
def update_button_list(selected_idx):
    if selected_idx is not None:
        out_id = 'div_%s' % selected_idx
        out_obj_list = [html.Div('ClickOutputs',id = out_id)]
        out_dep_obj = Output(component_id= 'click_msg', component_property='children')
        in_dep_obj = []
        for i in range(4):
            c_id = 'id_%s_%04d' % (selected_idx, i)
            out_obj_list += [html.Button('Hey %04d' % (i), id = c_id) ]
            in_dep_obj += [Input(component_id=c_id, component_property='n_clicks')]
        test_dash_app.callback(out_dep_obj, in_dep_obj)(fancy_button_adder)
        return out_obj_list
#test_dash_app.config['suppress_callback_exceptions']=True            

In [ ]:
show_app(test_dash_app)

In [ ]:

test_dash_app = dash.Dash(__name__, url_base_pathname='/', csrf_protect=False)
test_dash_app.layout = html.Div([dcc.RadioItems(id='item_list', 
                                                options = [dict(label = k, value = k) for k in ['Hey', 'Bob']]), 
                                 html.Div(id='button_list'),
                                 dcc.Location(id='url', refresh=False),
                                html.Div('No Clicks', id = 'click_msg')])
@test_dash_app.callback(
    Output(component_id='button_list', component_property='children'),
    [Input(component_id='item_list', component_property='value')]
)
def update_button_list(selected_idx):
    if selected_idx is not None:
        return [html.Div([html.Br(), 
                          dcc.Link('Hey %04d' % (i), 
                                   href = 'id_%s_%04d' % (selected_idx, i),
                                  style = {'color': '#1EAEDB', 'text-decoration': 'underline',
                                           'cursor': 'pointer'})]) for i in range(4)]

@test_dash_app.callback(
    Output(component_id='click_msg', component_property='children'),
    [Input(component_id='url', component_property='pathname')]
)
def update_click_msg(in_url):
    if in_url != '/':
        p_url = in_url.split('/')[-1]
        return p_url
    else:
        return 'No Clicks'

In [ ]:
show_app(test_dash_app)